In [1]:
import os
import re
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import tensorflow.keras as keras
import spacy

SEED = 42
np.random.seed(SEED)
AUTOTUNE = tf.data.AUTOTUNE

In [2]:
def convert_to_int(label):
    if label == "positive":
        return 1
    elif label == "negative":
        return 0
    
dataset = pd.read_csv(os.path.expanduser("~\Downloads\datasets\IMDB-Dataset.csv"))
dataset['sentiment'] = dataset['sentiment'].map(convert_to_int)
# dataset = pd.read_csv("/content/drive/MyDrive/IMDB-Dataset.csv")
train_ae = dataset.iloc[:44000, 0]
val_ae = dataset.iloc[44000:47000, 0]
test_ae = dataset.iloc[47000:, 0]

train_y = dataset.iloc[:44000, 1]
val_y = dataset.iloc[44000:47000, 1]
test_y = dataset.iloc[47000:, 1]

print(train_ae.head())
print("\n", train_y.head())

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
Name: review, dtype: object

 0    1
1    1
2    1
3    0
4    1
Name: sentiment, dtype: int64


# Tokenization

In [3]:
print(train_ae[1][:190])
html_re = re.compile(r'<[^>]+>')
train_ae = train_ae.replace(html_re, "", regex=True)
print(train_ae[1][:190])

A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the en
A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. 


In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(train_ae)
print("Word Indices: ")
print(dict(itertools.islice(tokenizer.index_word.items(), 20)), "...")
print(f"Word Count: {len(tokenizer.word_index)}")

Word Indices: 
{1: 'the', 2: 'and', 3: 'a', 4: 'of', 5: 'to', 6: 'is', 7: 'in', 8: 'it', 9: 'i', 10: 'this', 11: 'that', 12: 'was', 13: 'as', 14: 'for', 15: 'with', 16: 'movie', 17: 'but', 18: 'film', 19: 'on', 20: 'not'} ...
Word Count: 119224


In [5]:
MAX_NUM_TOKENS = 20000
tokenizer.num_words = MAX_NUM_TOKENS
train_ae = train_ae.apply(lambda x: tokenizer.texts_to_sequences([x])[0])
val_ae = val_ae.apply(lambda x: tokenizer.texts_to_sequences([x])[0])
test_ae = test_ae.apply(lambda x: tokenizer.texts_to_sequences([x])[0])

train_ae.head()

0    [26, 4, 1, 79, 2077, 44, 1059, 11, 98, 146, 38...
1    [3, 393, 119, 353, 1, 1350, 2947, 6, 51, 18582...
2    [9, 190, 10, 12, 3, 393, 94, 5, 1133, 54, 19, ...
3    [689, 221, 3, 232, 116, 3, 119, 426, 3595, 128...
4    [111, 7, 1, 54, 4, 287, 6, 3, 2074, 1438, 18, ...
Name: review, dtype: object

# Padding

In [6]:
print('sequence_length mean: ', train_ae.apply(lambda x: len(x)).mean())
print('sequence_length median: ', train_ae.apply(lambda x: len(x)).median())
print('sequence_length std deviation: ', np.sqrt(train_ae.apply(lambda x: len(x)).var()))
MAX_SEQUENCE_LENGTH = 300 # mean + std deviation*0.5

sequence_length mean:  224.09893181818182
sequence_length median:  169.0
sequence_length std deviation:  164.80941972297364


In [7]:
train_ae = keras.preprocessing.sequence.pad_sequences(train_ae, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
val_ae = keras.preprocessing.sequence.pad_sequences(val_ae, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
test_ae = keras.preprocessing.sequence.pad_sequences(test_ae, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
print(train_ae)

[[  26    4    1 ...  124 4081  485]
 [   3  393  119 ...    0    0    0]
 [   9  190   10 ...    0    0    0]
 ...
 [   1 3047    5 ...   15 2055    1]
 [ 528   10   26 ...    0    0    0]
 [   9  152   24 ...    0    0    0]]


# Word to Vector

In [8]:
word2vec_model = spacy.load("en_core_web_sm")

df_index_word = pd.Series(tokenizer.index_word) # Get all the words in the vocabulary
df_index_word = df_index_word[:MAX_NUM_TOKENS-1]
df_index_word = pd.Series(["temp_oov_word"]).append(df_index_word) # Add a temporary row for oov words
df_index_word = df_index_word.reset_index() # New column for token_id
df_index_word.columns = ['token_id', 'token']


df_index_word['word2vec'] = df_index_word.token.apply(lambda x: word2vec_model(x).vector) # Embedding for each word
df_index_word.at[0, "word2vec"] = np.zeros_like(df_index_word.at[0, "word2vec"]) # Replace the embedding for "temp_oov_word" with 0s
df_index_word.head()

,token_id,token,word2vec
0,0,temp_oov_word,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,the,"[-0.2032743, -0.4009009, -0.73730505, 0.536398..."
2,2,and,"[-0.9291044, -0.13172856, -0.814826, 0.6040276..."
3,3,a,"[0.02819018, 0.009864252, -0.095220566, 0.5522..."
4,4,of,"[-1.1193825, 0.18575507, -0.59173, -0.4724092,..."


In [9]:
embedding_matrix = np.array([vec for vec in df_index_word.word2vec.values])
print(embedding_matrix.shape) # MAX_NUM_TOKENS x vector dim 
embedding_matrix[0:3, :5]

(20000, 96)


array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.2032743 , -0.4009009 , -0.73730505,  0.5363982 ,  0.3764372 ],
       [-0.9291044 , -0.13172856, -0.814826  ,  0.6040276 , -0.67340827]],
      dtype=float32)

In [10]:
# MAX_NUM_TOKENS = 20000
# MAX_SEQUENCE_LENGTH = 300
EMBEDDING_DIM = embedding_matrix.shape[1] #96
embedding_layer = keras.layers.Embedding(
    input_dim=MAX_NUM_TOKENS,
    output_dim=EMBEDDING_DIM,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    input_length=MAX_SEQUENCE_LENGTH,
    mask_zero=True,
    trainable=False)

# test
embedding_result = embedding_layer(inputs=train_ae[0])
print(embedding_result.shape) # MAX_SEQUENCE_LENGTH x EMBEDDING_DIM
print("Embeddings of the first 5 words (each vector is restricted to size 5)")
print(train_ae[0][:5])
print(embedding_result[:5, :5]) 

train_ae = embedding_layer(inputs=train_ae)
val_ae = embedding_layer(inputs=val_ae)
test_ae = embedding_layer(inputs=test_ae)

(300, 96)
Embeddings of the first 5 words (each vector is restricted to size 5)
[  26    4    1   79 2077]
tf.Tensor(
[[ 0.49959356  0.704657    0.4497372   0.32913145  0.01701519]
 [-1.1193825   0.18575507 -0.59173    -0.4724092   0.04607756]
 [-0.2032743  -0.4009009  -0.73730505  0.5363982   0.3764372 ]
 [-0.33528453 -0.5446908   0.7285543  -0.51831293  0.10952553]
 [-0.39640257  0.09997782 -0.0515418   0.10987999  0.6886112 ]], shape=(5, 5), dtype=float32)


In [94]:
conv_encoder = keras.models.Sequential([
    keras.layers.Reshape([MAX_SEQUENCE_LENGTH, EMBEDDING_DIM, 1], input_shape=[MAX_SEQUENCE_LENGTH, EMBEDDING_DIM]),
    keras.layers.Conv2D(filters=96, kernel_size=(8, EMBEDDING_DIM), strides=2), # 
    # keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Activation('selu'),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=128, kernel_size=(32,1), strides=1),
    # keras.layers.MaxPool2D(pool_size=4),
    keras.layers.Activation('selu'),
    keras.layers.BatchNormalization(),
    
    # try with just another conv2D, pool, activation layer without using fully conv layers.
    keras.layers.Conv2D(filters=50, kernel_size=(116,1)), # fully convolutional layer, same kernel size as input size.
    # keras.layers.BatchNormalization(),
    # keras.layers.Conv2D(filters=50, kernel_size=(1,1)), # fully conv layer
    keras.layers.Reshape([50])
])
# conv_encoder.summary()
conv_decoder = keras.models.Sequential([
    # kernel size and strides are chosen such that we get 300x96 = 28800 = 180x160 size.
    keras.layers.Reshape([1,1,50], input_shape=[50]),
    keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='valid', input_shape=[1,1,50]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(32, kernel_size=16, strides=4, padding='valid', activation='selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(16, kernel_size=26, strides=2, padding='valid', activation='selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(1, kernel_size=(22, 1), strides=2, padding='valid', activation='linear'), 
    keras.layers.Reshape([MAX_SEQUENCE_LENGTH, EMBEDDING_DIM])
])
conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])
for l in conv_ae.layers:
    l.summary()

Model: "sequential_107"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_122 (Reshape)        (None, 300, 96, 1)        0         
_________________________________________________________________
conv2d_126 (Conv2D)          (None, 147, 1, 96)        73824     
_________________________________________________________________
activation_83 (Activation)   (None, 147, 1, 96)        0         
_________________________________________________________________
batch_normalization_174 (Bat (None, 147, 1, 96)        384       
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 116, 1, 128)       393344    
_________________________________________________________________
activation_84 (Activation)   (None, 116, 1, 128)       0         
_________________________________________________________________
batch_normalization_175 (Bat (None, 116, 1, 128)    

In [95]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, verbose=1, restore_best_weights=True,)
conv_ae.compile(optimizer='adam', loss='mse') # metrics='accuracy' is useful only when doing classification
conv_ae.fit(x=train_ae, y=train_ae, epochs=3, batch_size=256, validation_data=(val_ae, val_ae))

Epoch 1/3
172/172 [==============================] - 30s 165ms/step - loss: 0.2277 - val_loss: 0.2209
Epoch 2/3
172/172 [==============================] - 28s 161ms/step - loss: 0.2195 - val_loss: 0.2199
Epoch 3/3
172/172 [==============================] - 29s 166ms/step - loss: 0.2191 - val_loss: 0.2195


In [96]:
encoder_classifier = keras.models.clone_model(conv_encoder)
encoder_classifier.add(keras.layers.Dense(40, activation='selu', activity_regularizer='l2'))
encoder_classifier.add(keras.layers.Dense(20, activation='selu', activity_regularizer='l2'))
encoder_classifier.add(keras.layers.Dense(1, activation='sigmoid'))
encoder_classifier.summary()

Model: "sequential_107"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_122 (Reshape)        (None, 300, 96, 1)        0         
_________________________________________________________________
conv2d_126 (Conv2D)          (None, 147, 1, 96)        73824     
_________________________________________________________________
activation_83 (Activation)   (None, 147, 1, 96)        0         
_________________________________________________________________
batch_normalization_174 (Bat (None, 147, 1, 96)        384       
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 116, 1, 128)       393344    
_________________________________________________________________
activation_84 (Activation)   (None, 116, 1, 128)       0         
_________________________________________________________________
batch_normalization_175 (Bat (None, 116, 1, 128)    

In [97]:
encoder_classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
encoder_classifier.fit(x=train_ae, y=tf.convert_to_tensor(train_y), epochs=30, batch_size=256, validation_data=(val_ae, tf.convert_to_tensor(val_y)))

Epoch 1/30
172/172 [==============================] - 6s 29ms/step - loss: 2.0002 - accuracy: 0.5514 - val_loss: 1.7579 - val_accuracy: 0.5130
Epoch 2/30
172/172 [==============================] - 5s 28ms/step - loss: 1.5953 - accuracy: 0.5907 - val_loss: 1.5340 - val_accuracy: 0.5653
Epoch 3/30
172/172 [==============================] - 5s 28ms/step - loss: 1.3992 - accuracy: 0.6159 - val_loss: 1.3860 - val_accuracy: 0.5713
Epoch 4/30
172/172 [==============================] - 5s 28ms/step - loss: 1.3307 - accuracy: 0.6263 - val_loss: 1.3055 - val_accuracy: 0.6373
Epoch 5/30
172/172 [==============================] - 5s 28ms/step - loss: 1.2584 - accuracy: 0.6488 - val_loss: 1.1272 - val_accuracy: 0.6360
Epoch 6/30
172/172 [==============================] - 5s 28ms/step - loss: 1.0115 - accuracy: 0.6784 - val_loss: 1.1405 - val_accuracy: 0.5893
Epoch 7/30
172/172 [==============================] - 5s 28ms/step - loss: 0.7245 - accuracy: 0.7007 - val_loss: 0.7216 - val_accuracy: 0.6493

KeyboardInterrupt: 

In [ ]:
# https://github.com/tensorflow/models/issues/9706#issuecomment-875123676

recurrent_encoder = keras.models.Sequential([
    embedding_layer,
    keras.layers.GRU(300, return_sequences=True, input_shape=[MAX_SEQUENCE_LENGTH, EMBEDDING_DIM]),
    keras.layers.GRU(100, return_sequences=True),
    keras.layers.GRU(30)
])
recurrent_decoder = keras.models.Sequential([
    keras.layers.RepeatVector(MAX_SEQUENCE_LENGTH, input_shape=[30]),
    keras.layers.GRU(100, return_sequences=True),
    keras.layers.GRU(300, return_sequences=True),
    keras.layers.GRU(100, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(EMBEDDING_DIM, activation="sigmoid"))
])
recurrent_ae = keras.models.Sequential([recurrent_encoder, recurrent_decoder])

In [ ]:
for l in recurrent_ae.layers:
    l.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 96)           1920000   
_________________________________________________________________
gru (GRU)                    (None, 300, 300)          358200    
_________________________________________________________________
gru_1 (GRU)                  (None, 300, 100)          120600    
_________________________________________________________________
gru_2 (GRU)                  (None, 30)                11880     
Total params: 2,410,680
Trainable params: 490,680
Non-trainable params: 1,920,000
_________________________________________________________________
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
repeat_vector_3 (RepeatVecto (None, 300, 30)           0         
_______________

In [ ]:
recurrent_ae.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [ ]:
# recurrent_ae.fit(x=train_ae, y=embedding_layer(train_ae), epochs=5)